# Color Image Quantization
Reduce the number of colors in images (`lena.png`, `peppers.tif`) to 16 using K-means clustering, implemented from scratch. Log L2 norms per iteration and optionally visualize quantized images.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [96]:
def kmeans_quantization(image, k=16, max_iterations=100, visualize=True):
    """Perform K-means color quantization on an image.

    Args:
        image: Input image (H, W, 3) as float32 in [0, 1].
        k: Number of clusters (colors).
        max_iterations: Maximum number of iterations.
        visualize: If True, display quantized image per iteration.

    Returns:
        quantized_image: Image with k colors.
        l2_norms: List of L2 norms per iteration.
    """
    # TODO: Implement K-means clustering from scratch
    # 1. Reshape image to list of pixels (N, 3)
    pixels = image.reshape(-1, 3)
    N = pixels.shape[0]
    H, W = image.shape[:2]
    l2_norms = []
    distances = np.zeros((N, k))
    # 2. Initialize k centroids randomly
    centroids=pixels[np.random.choice(N, k, replace=False)].copy()
    new_centroids = centroids.copy()
    # 3. Assign pixels to nearest centroid (L2 norm)
    for i in range(max_iterations):
      for j in range(k):
        distances[: ,j] = np.sqrt((pixels[:, 0]-centroids[j, 0])**2+(pixels[:, 1]-centroids[j, 1])**2+(pixels[:, 2]-centroids[j, 2])**2)
      Assigned_pixels = np.argmin(distances, axis=1)
    # 4. Update centroids as mean of assigned pixels
      for j in range(k):
        new_centroids[j] = np.mean(pixels[Assigned_pixels == j], axis=0)
      if np.allclose(centroids, new_centroids):
        break
      centroids = new_centroids.copy()
    # 5. Log L2 norm per iteration
      L2_norm = 0.0
      for j in range(N):
          L2_norm += (pixels[j, 0] - centroids[Assigned_pixels[j], 0])**2 + (pixels[j, 1] - centroids[Assigned_pixels[j], 1])**2 + (pixels[j, 2] - centroids[Assigned_pixels[j], 2])**2

      l2 = np.sqrt(L2_norm)
      l2_norms.append(l2)
    # 6. Visualize if flag is enabled
      if visualize:
        quantized = centroids[Assigned_pixels].reshape(H, W, 3)
        plt.imshow(cv2.cvtColor(quantized, cv2.COLOR_BGR2RGB))
        plt.title(f'Iteration:{i+1},L2:{l2:.2f}')
        plt.axis('off')
        plt.show()
    final_quantized = centroids[Assigned_pixels].reshape(H, W, 3)
    return final_quantized, l2_norms

In [97]:
def calculate_l2_norm(original, quantized):
    """Calculate total L2 norm between original and quantized images.

    Args:
        original: Original image (H, W, 3) as float32 in [0, 1].
        quantized: Quantized image (H, W, 3) as float32 in [0, 1].

    Returns:
        l2_norm: Total L2 norm across all pixels.
    """
    # TODO: Compute L2 norm: sqrt(sum((R1-R2)^2 + (G1-G2)^2 + (B1-B2)^2))
    l2_norm = np.sqrt(np.sum((original - quantized)**2))
    return l2_norm


In [98]:
# Load images
lena_path = './images_CIQ/lena.png'

lena = cv2.imread(lena_path).astype(np.float32) / 255.0

# Parameters
k = 16
max_iterations = 100
visualize = True

In [ ]:
# Process images
lena_quantized, lena_l2_norms = kmeans_quantization(lena, k, max_iterations, visualize)

# Save L2 norms
with open('L2_norm_log.txt', 'w') as f:
    f.write('Lena L2 Norms:\n' + '\n'.join(map(str, lena_l2_norms)) + '\n')

# Calculate final L2 norms
lena_l2 = calculate_l2_norm(lena, lena_quantized)

# Visualize results
plt.imshow(cv2.cvtColor(lena_quantized, cv2.COLOR_BGR2RGB))
plt.title('Lena Quantized')
plt.axis('off')
plt.show()

print(f'Total L2 Norm: {lena_l2:.2f}')

# Save quantized images
cv2.imwrite('lena_quantized.png', (lena_quantized * 255).astype(np.uint8))